# Libraries

Here I'll check how to do some things efficiently

#### Choose best iteration

What we want to do is: given a dataset, and mlp value, find the best iteration (so when the highest NDCG@10 was achieved)\
Steps:
1. Find the file that contains the information of which is the best iteration
2. Read the best iteration
3. Read the file that contains the best iteration 

In [ ]:
data = 'fairbook'
mlp = 

In [ ]:
recs_location = 'results/'+data+str(fold)+'/recs/'

In [ ]:
recs_file = recs_location + algorithm+'_seed=42_e=25_bs='+str(batch_size)+'_lr=0$1_umlp='+mlp+'_imlp='+mlp+'_negratio=5_reg=0$001_sim=cosine_it=25.tsv'